### prerequisite is hashkey_create.ipynb

<https://cloud.google.com/firestore/docs/create-database-server-client-library#initialize>

In [1]:
import os, re, hmac
from base64 import b64encode, b64decode
from google.cloud import firestore
from google.cloud import secretmanager

In [2]:
PROJECT_ID = "practice-springml-ssnhash-py"
SECRET = "hash-key-64"
SALT = "eeba2d314a46c84ed4c3cab991d0359a"
COLLECTION = "hashed_socials"

In [3]:
def access_hashkey(project_id, secret_id, version_id):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    response = client.access_secret_version(request={"name": name})
    payload = response.payload.data
    return b64decode(payload)

In [4]:
def hash_ssn(ssn, key, salt):
    norm_ssn = ssn.strip().replace('-', '')
    if not re.match(r'[0-9]{9}', norm_ssn):
        raise ValueError(f"Normalized SSN from {norm_ssn} is not a 9 digit number")
    salt = salt.encode('utf-8')
    mac = hmac.new(key, msg=salt, digestmod='sha256')
    mac.update(norm_ssn.encode('utf-8'))
    return mac.hexdigest()

In [5]:
hashkey = access_hashkey(PROJECT_ID, SECRET, "latest")

db = firestore.Client(project = PROJECT_ID)

for ssn in open("fake_socials.txt"):

    hashed_ssn = hash_ssn(ssn, hashkey, SALT)
    db.collection(COLLECTION).document(hashed_ssn).set({u'exists': True})
